In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, matthews_corrcoef
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

In [2]:
loans = pd.read_csv('Assignment2Data.csv', low_memory=False, header=0)
#loans = pd.read_csv("./Assignment2Data.csv", low_memory=False, header=0)
loans.head(n=10)

,acc_now_delinq,acc_open_past_24mths,all_util,annual_inc,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,collections_12_mths_ex_med,delinq_amnt,...,total_cu_tl,total_il_high_credit_limit,total_pymnt,total_pymnt_inv,total_rec_int,total_rec_late_fee,total_rec_prncp,total_rev_hi_lim,delinq_2yrs,bad_loans
0,0.0,3.0,1.0,52000.0,25099.0,30359.0,0.5,0.0,0.0,0.0,...,2.0,0.0,3011.577285,3011.58,11.58,0.0,3000.00,31000.0,0.0,0
1,0.0,2.0,47.0,55000.0,2222.0,10551.0,52.0,0.0,0.0,0.0,...,1.0,27751.0,5013.306667,5013.31,13.31,0.0,5000.00,33800.0,0.0,0
2,0.0,3.0,86.0,51000.0,4541.0,2498.0,0.1,0.0,0.0,0.0,...,0.0,24001.0,1032.903619,1032.90,32.90,0.0,1000.00,2500.0,0.0,0
3,0.0,2.0,41.0,62000.0,18520.0,18310.0,51.2,0.0,0.0,0.0,...,0.0,21976.0,20009.805556,19984.79,9.81,0.0,20000.00,39500.0,0.0,0
4,0.0,11.0,60.0,30000.0,5990.0,7898.0,7.1,0.0,0.0,0.0,...,5.0,62319.0,24334.736554,24334.74,334.74,0.0,24000.00,12000.0,0.0,0
5,0.0,3.0,32.0,58000.0,16129.0,10282.0,6.5,0.0,0.0,0.0,...,0.0,68112.0,265.850000,265.30,223.63,15.0,27.22,11000.0,1.0,1
6,0.0,2.0,57.0,48000.0,2855.0,7459.0,25.4,0.0,0.0,0.0,...,0.0,27697.0,0.000000,0.00,0.00,0.0,0.00,12200.0,1.0,1
7,0.0,21.0,32.0,186500.0,2443.0,107477.0,18.8,0.0,0.0,0.0,...,0.0,57714.0,20117.246667,20117.25,117.25,0.0,20000.00,139400.0,0.0,0
8,0.0,7.0,58.0,12000.0,48449.0,100.0,96.0,0.0,0.0,0.0,...,0.0,0.0,15424.086919,15424.09,424.09,0.0,15000.00,12400.0,1.0,0
9,0.0,4.0,74.0,54000.0,11336.0,6635.0,44.7,0.0,0.0,0.0,...,3.0,34050.0,2505.296389,2505.30,5.30,0.0,2500.00,12000.0,0.0,0


In [3]:
# Check how many good and bad loans there are
loans['bad_loans'].value_counts()

0    2516
1     476
Name: bad_loans, dtype: int64

### Oversample Bad Loans so that it is proportionate

In [4]:
# Class count
count_class_0, count_class_1 = loans['bad_loans'].value_counts()

# Divide by class
df_class_0 = loans[loans['bad_loans'] == 0]
df_class_1 = loans[loans['bad_loans'] == 1]

df_class_1_over = df_class_1.sample(count_class_0, replace=True, random_state=1234)

# Address unblanced data issue: Oversampling on data samples with rare outputs (which is '1' in this example)
df_class_1_over = df_class_1.sample(count_class_0, replace=True, random_state=1234)
loans = pd.concat([df_class_1_over, df_class_0], axis=0)

loans['bad_loans'].value_counts()

1    2516
0    2516
Name: bad_loans, dtype: int64

### Split Data 80/20

In [5]:
X = loans.drop('bad_loans', axis=1) # Here no need to set inplace=True
Y = loans['bad_loans']

In [6]:
validation_size = 0.2
seed = 1234
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=validation_size, random_state=seed)

<h1 style="color:blue">FULL MODEL</h1>

In [7]:
def get_results(predictions):
    print("Accuracy score is: " + str(accuracy_score(Y_test, predictions)))
    print("F1_score is: " + str(f1_score(Y_test, predictions)))
    print("MCC: ", matthews_corrcoef(Y_test, predictions))
    print("Confusion Matrix")
    print(confusion_matrix(Y_test, predictions))
    return

## Logistic Regression

In [8]:
#Logistic Regression, use default param values then tune in subsequent parts
def logisticRegression(X_train, Y_train, X_test, Y_test):
    lg = LogisticRegression(random_state=seed, solver='liblinear')
    lg.fit(X_train, Y_train)
    #print(lg.get_params) #print this to see possible params to tune
    predictions = lg.predict(X_test)
    get_results(predictions)
    return

logisticRegression(X_train, Y_train, X_test, Y_test)

Accuracy score is: 0.9225422045680238
F1_score is: 0.9277777777777779
MCC:  0.8533127404720492
Confusion Matrix
[[428  74]
 [  4 501]]


In [9]:
LR_clf = LogisticRegression(random_state=seed, solver='liblinear', C=464.15888336127773, penalty='l2' )
LR_model = LR_clf.fit(X_train, Y_train)
predictions = LR_model.predict(X_test)
get_results(predictions)
#same result as before

Accuracy score is: 0.9275074478649454
F1_score is: 0.9319664492078287
MCC:  0.8617370451537003
Confusion Matrix
[[434  68]
 [  5 500]]


### Full Model - Results for Logistic Regression

Logistic Regression without GridSearchCV: 92.3% accuracy, 0.853 MCC

Logistic Regression with GridSearchCV: 92.8% accuracy, 0.862 MCC 

Slight increase in accuracy and MCC

## Decision Tree

In [10]:
def decisionTree(X_train, Y_train, X_test, Y_test):
    dtc = DecisionTreeClassifier(random_state=seed)
    dtc.fit(X_train, Y_train)
    predictions = dtc.predict(X_test)
    get_results(predictions)
    return
decisionTree(X_train, Y_train, X_test, Y_test)

Accuracy score is: 0.9602780536246276
F1_score is: 0.9619047619047619
MCC:  0.9234567649066872
Confusion Matrix
[[462  40]
 [  0 505]]


In [11]:
#Tuned Model 
DT_clf = DecisionTreeClassifier(random_state=seed, max_depth = 9, min_samples_split= 0.1, \
                             min_samples_leaf= 1, criterion='gini')
DT_model = DT_clf.fit(X_train, Y_train)
predictions = DT_model.predict(X_test)
get_results(predictions)

Accuracy score is: 0.9285004965243296
F1_score is: 0.9319470699432891
MCC:  0.860882316689042
Confusion Matrix
[[442  60]
 [ 12 493]]


### Full Model - Results for Decision Tree

Decision Tree without GridSearchCV: 96.0% accuracy, 0.923 MCC

Decision tree with GridSearchCV: 92.9% accuracy, 0.861 MCC


The default hyperparameter values perform better than the best param found in the param grid set up for in GridSearchCV. 

## XGBClassifier

In [12]:
##XGBOOST

def xgBoost(X_train, Y_train, X_test, Y_test):
    xgbc = XGBClassifier(random_state=seed)
    xgbc.fit(X_train, Y_train)
    predictions = xgbc.predict(X_test)
    get_results(predictions)
    return

xgBoost(X_train, Y_train, X_test, Y_test)

Accuracy score is: 0.9404170804369414
F1_score is: 0.943609022556391
MCC:  0.8859081552873652
Confusion Matrix
[[445  57]
 [  3 502]]


In [13]:
XGB_clf = XGBClassifier(random_state=seed, max_depth = 6 , learning_rate = 0.1)
XGB_model = XGB_clf.fit(X_train, Y_train)
predictions = XGB_model.predict(X_test)
get_results(predictions)

Accuracy score is: 0.9493545183714002
F1_score is: 0.9519321394910463
MCC:  0.9033264058721377
Confusion Matrix
[[451  51]
 [  0 505]]


### Full Model - Results for XGBoost

XGBoost without GridSearchCV:  94.0% accuracy, 0.886 MCC

XGBoost with GridSearchCV: 94.9% accuracy, 0.903 MCC 

We see a slight increase in accuracy rate and MCC value. Since accuracy rates are already high to begin with before hyperparameter tuning, we would expect to see minimal increase in accuracy and MCC values as well.

## Question 3 

**Which loan attributes do you believe are the most informative? Please use your knowledge and intuition to choose 10 of the attributes available, and explain why you chose these attributes. Let’s call this the REDUCED-MODEL.**


1. Select 10 variables

2. Test out model without tuning

3. Test out model with hyperparameter tuning. 

In [14]:
#cols = ['annual_inc','avg_cur_bal', 'dti', 'int_rate', 'loan_amnt',\
       #'max_bal_bc', 'mo_sin_old_il_acct', 'pct_tl_nvr_dlq', 'mort_acc', 'num_sats']

cols= ['annual_inc' ,'avg_cur_bal', 'dti','mths_since_last_delinq', 'mort_acc',\
'num_sats', 'open_acc_6m','total_bc_limit','total_pymnt' ,'total_rec_prncp']

X_train2 = X_train[cols]
X_test2 = X_test[cols]    



## Logistic Regression

In [15]:
logisticRegression(X_train2, Y_train, X_test2, Y_test)

Accuracy score is: 0.9155908639523337
F1_score is: 0.9210770659238625
MCC:  0.8385769519352704
Confusion Matrix
[[426  76]
 [  9 496]]


In [16]:
LR_clf = LogisticRegression(random_state=seed, solver='liblinear', C=1,  penalty='l2' )
LR_model = LR_clf.fit(X_train2, Y_train)
predictions = LR_model.predict(X_test2)
get_results(predictions)
#same result as before

Accuracy score is: 0.9155908639523337
F1_score is: 0.9210770659238625
MCC:  0.8385769519352704
Confusion Matrix
[[426  76]
 [  9 496]]


### Reduced Model - Results for Logistic Regression ###

Logistic Regression without GridSearchCV: 91.6% accuracy, 0.839 MCC

Logistic Regression with GridSearchCV: 91.6% accuracy, 0.839 MCC

Both show the same accuracy and MCC. No improvement from the GridSearchCV.


## Decision Tree

In [17]:
decisionTree(X_train2, Y_train, X_test2, Y_test)

Accuracy score is: 0.9543197616683218
F1_score is: 0.9562737642585551
MCC:  0.9117954268367235
Confusion Matrix
[[458  44]
 [  2 503]]


In [18]:
#Decision Tree GridSearchCV
#DO NOT RUN THIS, see below for best params
dtc = DecisionTreeClassifier(random_state=seed)

#hyperparameters to tune
max_depth = [i for i in range(1,11)]
min_samples_split = [i*0.1 for i in range(1,11)]
min_samples_leaf = [1, 5, 10, 20, 50, 100]
criterion= ['gini', 'entropy']

# Create hyperparameter options
hyperparameters = dict(max_depth=max_depth, min_samples_leaf=min_samples_leaf,min_samples_split=min_samples_split, criterion=criterion)
clf = GridSearchCV(dtc, hyperparameters, cv=5, verbose=0)

best_model = clf.fit(X_train2, Y_train)
print(best_model.best_params_)

{'criterion': 'entropy', 'max_depth': 3, 'min_samples_leaf': 100, 'min_samples_split': 0.1}


In [19]:
#Based on GridSearchCV
DT_clf = DecisionTreeClassifier(random_state=seed, max_depth = 3, min_samples_split=0.1,\
                             min_samples_leaf= 100, criterion='entropy')
DT_model = DT_clf.fit(X_train2, Y_train)
predictions = DT_model.predict(X_test2)
get_results(predictions)

Accuracy score is: 0.8967229394240318
F1_score is: 0.9018867924528301
MCC:  0.7973264681686234
Confusion Matrix
[[425  77]
 [ 27 478]]


### Reduced Model - Results for Decision Tree ###

Decision tree without GridSearchCV : 95.4% accuracy, 0.912 MCC

Decision tree with GridSearchCV: 89.7% accuracy, 0.797 MCC


The default hyperparameter values perform better than the best param found in the param grid set up for in GridSearchCV. 

In [20]:
xgBoost(X_train2, Y_train, X_test2, Y_test)

Accuracy score is: 0.9304865938430983
F1_score is: 0.9343339587242027
MCC:  0.866304400246787
Confusion Matrix
[[439  63]
 [  7 498]]


In [21]:
XGB_clf = XGBClassifier(random_state=seed, max_depth = 9 , learning_rate = 0.1)
XGB_model = XGB_clf.fit(X_train2, Y_train)
predictions = XGB_model.predict(X_test2)
get_results(predictions)

Accuracy score is: 0.9553128103277061
F1_score is: 0.957345971563981
MCC:  0.9142618241433023
Confusion Matrix
[[457  45]
 [  0 505]]


### Reduced Model - Results for XGBoost ###

XGBoost without GridSearchCV:  93.0% accuracy, 0.866 MCC

XGBoost with GridSearchCV: 95.5% accuracy, 0.914 MCC 

We see an increase in accuracy rates and MCC value. Since accuracy rates are already high to begin with before hyperparameter tuning, we would expect to see minimal increase in accuracy and MCC values as well.

<h1 style="color:blue">LASSO-REDUCED MODEL</h1>

In [22]:
# Import libraries
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn import tree
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")

In [23]:
loans = pd.read_csv('Assignment2Data.csv', low_memory=False, header=0)
#loans = pd.read_csv("./Assignment2Data.csv", low_memory=False, header=0)
loans.head()

,acc_now_delinq,acc_open_past_24mths,all_util,annual_inc,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,collections_12_mths_ex_med,delinq_amnt,...,total_cu_tl,total_il_high_credit_limit,total_pymnt,total_pymnt_inv,total_rec_int,total_rec_late_fee,total_rec_prncp,total_rev_hi_lim,delinq_2yrs,bad_loans
0,0.0,3.0,1.0,52000.0,25099.0,30359.0,0.5,0.0,0.0,0.0,...,2.0,0.0,3011.577285,3011.58,11.58,0.0,3000.0,31000.0,0.0,0
1,0.0,2.0,47.0,55000.0,2222.0,10551.0,52.0,0.0,0.0,0.0,...,1.0,27751.0,5013.306667,5013.31,13.31,0.0,5000.0,33800.0,0.0,0
2,0.0,3.0,86.0,51000.0,4541.0,2498.0,0.1,0.0,0.0,0.0,...,0.0,24001.0,1032.903619,1032.90,32.90,0.0,1000.0,2500.0,0.0,0
3,0.0,2.0,41.0,62000.0,18520.0,18310.0,51.2,0.0,0.0,0.0,...,0.0,21976.0,20009.805556,19984.79,9.81,0.0,20000.0,39500.0,0.0,0
4,0.0,11.0,60.0,30000.0,5990.0,7898.0,7.1,0.0,0.0,0.0,...,5.0,62319.0,24334.736554,24334.74,334.74,0.0,24000.0,12000.0,0.0,0


In [24]:
# Split the dataset
X = loans.drop('bad_loans', axis=1) # Here no need to set inplace=True
y = loans['bad_loans']

X_train_noscale, X_valid_noscale, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=1234)

In [25]:
# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_noscale)
X_valid = scaler.transform(X_valid_noscale)

X_train = pd.DataFrame(X_train, index=X_train_noscale.index, columns=X_train_noscale.columns)
X_valid = pd.DataFrame(X_valid, index=X_valid_noscale.index, columns=X_valid_noscale.columns)

In [26]:
# Initialize logistic regression model with L1 penalty: Lasso model
model = LogisticRegression(fit_intercept=True, penalty='l1', random_state=1234)

In [27]:
# A list of regularization paramaters/lambdas
lbd = np.logspace(-3, 3, 100)

In [28]:
# Find lambdas that limit to exactly 10 coefficients
model = LogisticRegression(fit_intercept=True, penalty='l1', random_state=1234)

coefs = [] 
parameters = []

# Train the model with different regularisation strengths
for a in lbd:
    model.set_params(C=1/a) # Note that C is inverse value of regularization factor lambda
    model.fit(X_train, y_train)
    coefficients = model.coef_[0]
    count = len(list(filter(lambda f: f!=0, coefficients)))
    
    if count == 10:
        coefs.append(coefficients)
        parameters.append(a)  

In [29]:
max_accuracy = 0
best_lambda = 0

for a in parameters:
    model.set_params(C=1/a) # Note that C is inverse value of regularization factor lambda
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)
    accuracy = accuracy_score(y_valid, y_pred)
    
    if accuracy >= max_accuracy:
        max_accuracy = accuracy
        best_lambda = a

In [30]:
best_lambda

30.538555088334185

In [31]:
# Logistic regression model:
lasso = LogisticRegression(fit_intercept=True, penalty='l1', C=1/best_lambda, random_state=1234)
lasso_model = lasso.fit(X_train, y_train)

# Get predicted labels for validation data
y_pred = lasso_model.predict(X_valid)

# Create confusion matrix of validation data
cm = confusion_matrix(y_valid, y_pred)
TN, FP, FN, TP = cm.ravel()
print("Lasso model: \n")
print("Confusion Matrix: \n", cm)
print("True negative: %s, False positive: %s, False negative: %s, True positive: %s" %(TN, FP, FN, TP))

# Performance of model on validation data
print("Accuracy: ", accuracy_score(y_valid, y_pred))
print("Sensitivity: ", recall_score(y_valid, y_pred))
print("Precision: ", precision_score(y_valid, y_pred))
print("MCC: ", matthews_corrcoef(y_valid, y_pred))

Lasso model: 

Confusion Matrix: 
 [[730  19]
 [ 98  51]]
True negative: 730, False positive: 19, False negative: 98, True positive: 51
Accuracy:  0.8697104677060133
Sensitivity:  0.3422818791946309
Precision:  0.7285714285714285
MCC:  0.43975650303673863


In [32]:
lasso_coefficients = lasso_model.coef_[0]
lasso_coefficients

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.28425555,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.03167739,  0.        ,  0.        ,
        0.08794408,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.67901033,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.02297276,  0.        ,
        0.        , -0.05158867,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        , -0.52446019, -1.0063375 ,
       -0.23866499,  0.        ,  0.        ,  0.        ,  0.00

In [33]:
index = [i for i in range(len(lasso_coefficients)) if lasso_coefficients[i]!=0]
index

[15, 22, 25, 46, 53, 56, 63, 64, 65, 69]

In [34]:
feature_full_list = list(loans.drop('bad_loans', axis=1).columns)
feature_full_list

['acc_now_delinq',
 'acc_open_past_24mths',
 'all_util',
 'annual_inc',
 'avg_cur_bal',
 'bc_open_to_buy',
 'bc_util',
 'chargeoff_within_12_mths',
 'collections_12_mths_ex_med',
 'delinq_amnt',
 'dti',
 'inq_fi',
 'inq_last_12m',
 'inq_last_6mths',
 'installment',
 'int_rate',
 'last_pymnt_amnt',
 'loan_amnt',
 'max_bal_bc',
 'mo_sin_old_il_acct',
 'mo_sin_old_rev_tl_op',
 'mo_sin_rcnt_rev_tl_op',
 'mo_sin_rcnt_tl',
 'mort_acc',
 'mths_since_last_delinq',
 'mths_since_rcnt_il',
 'num_accts_ever_120_pd',
 'num_actv_bc_tl',
 'num_actv_rev_tl',
 'num_bc_sats',
 'num_bc_tl',
 'num_il_tl',
 'num_op_rev_tl',
 'num_rev_accts',
 'num_rev_tl_bal_gt_0',
 'num_sats',
 'num_tl_30dpd',
 'num_tl_90g_dpd_24m',
 'num_tl_op_past_12m',
 'open_acc',
 'open_acc_6m',
 'open_act_il',
 'open_il_12m',
 'open_il_24m',
 'open_rv_12m',
 'open_rv_24m',
 'out_prncp',
 'out_prncp_inv',
 'pct_tl_nvr_dlq',
 'percent_bc_gt_75',
 'pub_rec_bankruptcies',
 'revol_bal',
 'revol_util',
 'tax_liens',
 'tot_coll_amt',
 'tot

In [35]:
selected_feature = [feature_full_list[j] for j in index]
selected_feature

['int_rate',
 'mo_sin_rcnt_tl',
 'mths_since_rcnt_il',
 'out_prncp',
 'tax_liens',
 'tot_hi_cred_lim',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_int',
 'delinq_2yrs']

In [36]:
acc = []
mcc = []
kf = KFold(n_splits=5)
for train_index, test_index in kf.split(X_train):
    lasso = LogisticRegression(fit_intercept=True, penalty='l1', C=1/best_lambda, random_state=1234)
    lasso_model = lasso.fit(X_train.iloc[train_index], y_train.iloc[train_index])
    predictions = lasso_model.predict(X_train.iloc[test_index])
    acc.append(accuracy_score(y_train.iloc[test_index], predictions))
    mcc.append(matthews_corrcoef(y_train.iloc[test_index], predictions))
    

In [37]:
from statistics import mean
print("average accuracy is: " + str(mean(acc)))
print("average mcc is: " + str(mean(mcc)))

average accuracy is: 0.8648605131835881
average mcc is: 0.39411665108756716
